In [1]:
from xg.utils import read_all_files,read_main_foot_data
from xg.features_new import *

In [2]:
df = read_file("Output/Data.csv")

c:\Users\Gabriel\Desktop\TCCv2\xg\utils.py:6: DtypeWarning: Columns (125,131,172,227,236) have mixed types. Specify dtype option on import or set low_memory=False.
  df_shots = pd.read_csv(path)


In [5]:
main_foot = read_main_foot_data("Output/main_foot.json")

In [6]:
STATSBOMB = get_statsbomb_features(df)

In [7]:
STATSBOMB['distance_to_goal'] = STATSBOMB['location_shot'].apply(lambda x: distance_to_goal(x))
STATSBOMB['angle_to_goal'] = STATSBOMB['location_shot'].apply(lambda x : angle_to_goal(x))
STATSBOMB['shot_body_part_mf'] = STATSBOMB.apply(lambda x : is_shot_with_main_foot(x['player_shot'],x['shot_body_part_shot'],main_foot),axis=1)
STATSBOMB['goal'] = STATSBOMB['shot_outcome_shot'].apply(lambda x : is_goal(x))
STATSBOMB['goalkeeper_distance_to_goal']  = STATSBOMB['shot_freeze_frame_shot'].apply(lambda x : goalkeeper_distance_to_goal(x))
STATSBOMB['goalkeeper_distance_to_shot']  = STATSBOMB.apply(lambda x : distance_goalkeeper_to_player(x['location_shot'],x['shot_freeze_frame_shot']),axis=1)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_4076\1049424894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STATSBOMB['distance_to_goal'] = STATSBOMB['location_shot'].apply(lambda x: distance_to_goal(x))
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_4076\1049424894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STATSBOMB['angle_to_goal'] = STATSBOMB['location_shot'].apply(lambda x : angle_to_goal(x))
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_4076\1049424894.py:3: SettingWithCopyWarning: 
A va

In [8]:
pa = STATSBOMB.apply(lambda x : players_between_goal(x['location_shot'],x['shot_freeze_frame_shot']),axis=1)
STATSBOMB['players_between_shot_and_goal'] = [p[0] for p in pa]
STATSBOMB['players_in_radius'] = [p[1] for p in pa]

KeyboardInterrupt: 

In [ ]:
STATSBOMB['goalkeeper_in_goal_pct'] = STATSBOMB.apply(lambda x : goalkeeper_between_goal(x['location_shot'],x['shot_freeze_frame_shot']),axis=1)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_9688\2671178545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STATSBOMB['goalkeeper_in_goal_pct'] = STATSBOMB.apply(lambda x : goalkeeper_between_goal(x['location_shot'],x['shot_freeze_frame_shot']),axis=1)


In [ ]:
STATSBOMB['goalkeeper_in_goal'] = STATSBOMB['goalkeeper_in_goal_pct'] > 0

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_9688\43780980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STATSBOMB['goalkeeper_in_goal'] = STATSBOMB['goalkeeper_in_goal_pct'] > 0


In [ ]:
STATSBOMBc = STATSBOMB.copy()

In [ ]:
STATSBOMBc['players_behind_ball'] = STATSBOMBc.apply(lambda x : number_of_opp_player_after_ball_line(x['location_shot'],x['shot_freeze_frame_shot']),axis=1)

In [ ]:
fillna = ["shot_open_goal_shot","shot_one_on_one_shot","under_pressure_shot","pass_cross_pass","shot_first_time_shot"]
fill_not_pass = ["pass_height_pass",]

In [ ]:
STATSBOMBc[fillna] = STATSBOMBc[fillna].fillna(False)
STATSBOMBc[fill_not_pass] = STATSBOMBc[fill_not_pass].fillna("Not pass")
STATSBOMBc["goalkeeper_position"] = STATSBOMBc["goalkeeper_position"].fillna("Not in Goal")
STATSBOMBc['under_pressure'] = STATSBOMBc['players_in_radius'] >= 1

In [ ]:
STATSBOMBc['x'] = STATSBOMBc['location_shot'].apply(lambda x : x[0])
STATSBOMBc['y'] = STATSBOMBc['location_shot'].apply(lambda x : x[1])

In [14]:
def split_data(df):
    
    """Given dataframe, separate in train and test split."""
    LaLigaFilter = (df['league'] == 'La Liga') & (df['season'] == '2015/2016')
    Xtrain = df[~LaLigaFilter].copy()
    Xtest = df[LaLigaFilter].copy()
    
    return Xtrain, Xtest

In [15]:
Xtrain,Xtest = split_data(STATSBOMBc)

In [16]:
Xtrain.columns

Index(['location_shot', 'shot_first_time_shot', 'shot_technique_shot',
       'shot_one_on_one_shot', 'shot_open_goal_shot', 'play_pattern_shot',
       'player_shot', 'season', 'team_shot', 'league',
       'shot_statsbomb_xg_shot', 'position_shot', 'pass_type_pass',
       'pass_cross_pass', 'shot_body_part_shot', 'pass_height_pass',
       'pass_technique_pass', 'goalkeeper_position', 'shot_freeze_frame_shot',
       'under_pressure_shot', 'pass_length_pass', 'play_before_type',
       'shot_outcome_shot', 'distance_to_goal', 'angle_to_goal',
       'shot_body_part_mf', 'goal', 'goalkeeper_distance_to_goal',
       'goalkeeper_distance_to_shot', 'players_between_shot_and_goal',
       'players_in_radius', 'goalkeeper_in_goal_pct', 'goalkeeper_in_goal',
       'players_behind_ball', 'under_pressure', 'x', 'y'],
      dtype='object')

In [17]:
Xtrain.to_csv("Output/Train.csv",index=False)

In [18]:
Xtest.to_csv("Output/Test.csv",index=False)